# 数据源与分析目的

数据源：https://tianchi.aliyun.com/dataset/97652

分析目的：分析每个客户的购物篮子，挖掘商品之间的关联关系，为商家提供更好的商品推荐。

# 导入库和数据

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import seaborn as sns 
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings 
warnings.filterwarnings('ignore')

In [2]:
import os
file_path = os.path.join(os.getcwd(),'order.csv')
data = pd.read_csv(file_path, encoding='gbk')
data.head()

,订单日期,年份,订单数量,产品ID,客户ID,交易类型,销售区域ID,销售大区,国家,区域,产品类别,产品型号名称,产品名称,产品成本,利润,单价,销售金额
0,2016/1/1,2016,1,528,14432BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
1,2016/1/2,2016,1,528,18741BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
2,2016/1/2,2016,1,528,27988BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
3,2016/1/5,2016,1,528,25710BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
4,2016/1/6,2016,1,528,14999BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0


In [3]:
data.shape

(60398, 17)

In [4]:
data.isnull().sum()

订单日期      0
年份        0
订单数量      0
产品ID      0
客户ID      0
交易类型      0
销售区域ID    0
销售大区      0
国家        0
区域        0
产品类别      0
产品型号名称    0
产品名称      0
产品成本      0
利润        0
单价        0
销售金额      0
dtype: int64

# 数据预处理

In [5]:
df = data.groupby('客户ID')['产品名称'].agg(list).reset_index()
df.head()

,客户ID,产品名称
0,13021BA,"[垒球, 三角网架, 棒球服, 头盔, 软式棒球, 棒球手套, 棒球手套, 软式棒球]"
1,13022BA,"[软式棒球, 硬式棒球, 球棒与球棒袋, 球棒与球棒袋, 球棒与球棒袋, 球棒与球棒袋, 软..."
2,13023BA,"[软式棒球, 垒球, 头盔, 软式棒球]"
3,13024BA,"[垒球, 帽子, 软式棒球, 球棒与球棒袋, 球棒与球棒袋, 棒球手套, 棒球手套, 棒球手..."
4,13025BA,"[软式棒球, 软式棒球, 垒球, 三角网架, 头盔, 头盔]"


In [6]:
# 去除重复的产品项，并且使用pd.unique(x)保留原始顺序
df['产品项'] = df['产品名称'].apply(lambda x: ','.join(pd.unique(x)))
df.head()

,客户ID,产品名称,产品项
0,13021BA,"[垒球, 三角网架, 棒球服, 头盔, 软式棒球, 棒球手套, 棒球手套, 软式棒球]","垒球,三角网架,棒球服,头盔,软式棒球,棒球手套"
1,13022BA,"[软式棒球, 硬式棒球, 球棒与球棒袋, 球棒与球棒袋, 球棒与球棒袋, 球棒与球棒袋, 软...","软式棒球,硬式棒球,球棒与球棒袋,三角网架,帽子,棒球服,头盔"
2,13023BA,"[软式棒球, 垒球, 头盔, 软式棒球]","软式棒球,垒球,头盔"
3,13024BA,"[垒球, 帽子, 软式棒球, 球棒与球棒袋, 球棒与球棒袋, 棒球手套, 棒球手套, 棒球手...","垒球,帽子,软式棒球,球棒与球棒袋,棒球手套"
4,13025BA,"[软式棒球, 软式棒球, 垒球, 三角网架, 头盔, 头盔]","软式棒球,垒球,三角网架,头盔"


In [7]:
# 构建双重列表结构
transactions = [i.split(',') for i in df['产品项']] # 按逗号分隔字符串
transactions

[['垒球', '三角网架', '棒球服', '头盔', '软式棒球', '棒球手套'],
 ['软式棒球', '硬式棒球', '球棒与球棒袋', '三角网架', '帽子', '棒球服', '头盔'],
 ['软式棒球', '垒球', '头盔'],
 ['垒球', '帽子', '软式棒球', '球棒与球棒袋', '棒球手套'],
 ['软式棒球', '垒球', '三角网架', '头盔'],
 ['软式棒球', '垒球', '棒球手套'],
 ['软式棒球', '球棒与球棒袋', '垒球'],
 ['软式棒球', '三角网架', '头盔', '棒球手套', '垒球'],
 ['垒球', '头盔', '软式棒球', '球棒与球棒袋', '棒球手套'],
 ['软式棒球', '垒球', '头盔', '棒球手套'],
 ['软式棒球', '垒球', '帽子'],
 ['软式棒球', '垒球', '棒球服'],
 ['头盔', '棒球手套'],
 ['头盔', '棒球手套'],
 ['头盔', '棒球手套', '击打手套'],
 ['打击T座', '三角网架', '软式棒球'],
 ['棒球手套', '软式棒球'],
 ['垒球', '软式棒球', '三角网架'],
 ['软式棒球', '垒球', '头盔', '球棒与球棒袋', '三角网架'],
 ['头盔', '袜子', '皮带', '棒球服', '球网', '击打手套', '球棒与球棒袋', '棒球手套'],
 ['软式棒球', '三角网架'],
 ['装备包', '软式棒球', '三角网架'],
 ['软式棒球', '棒球手套'],
 ['棒球手套', '棒球服', '三角网架'],
 ['棒球手套', '垒垫'],
 ['软式棒球', '捕手护具', '三角网架', '棒球服', '垒球'],
 ['软式棒球', '垒球', '装备包', '击打手套', '棒球手套'],
 ['软式棒球', '装备包', '帽子', '棒球服', '垒球', '棒球手套'],
 ['软式棒球', '垒球', '棒球手套'],
 ['垒球', '球棒与球棒袋', '软式棒球', '棒球服', '头盔'],
 ['软式棒球', '垒球', '帽子'],
 ['软式棒球', '棒球手套', '球棒与球棒袋', '垒球'],
 ['软式棒球

# 关联规则分析

In [8]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

df_te = pd.DataFrame(te_ary, columns= te.columns_)
df_te.head()

,三角网架,击打手套,垒垫,垒球,头盔,帽子,打击T座,捕手护具,棒球手套,棒球服,球棒与球棒袋,球网,皮带,硬式棒球,袜子,装备包,软式棒球
0,True,False,False,True,True,False,False,False,True,True,False,False,False,False,False,False,True
1,True,False,False,False,True,True,False,False,False,True,True,False,False,True,False,False,True
2,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,True
4,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True


In [9]:
df_te.shape

(18484, 17)

In [10]:
# 计算所有项集的支持度
support = df_te.mean()
# 将支持度列表转为DataFrame
df_support = pd.DataFrame(support, columns=['support']).sort_values(by='support', ascending=False)
df_support

,support
棒球手套,0.459316
硬式棒球,0.346083
头盔,0.322441
球棒与球棒袋,0.246051
软式棒球,0.221218
棒球服,0.172690
垒球,0.115938
帽子,0.115343
三角网架,0.114153
击打手套,0.074443


In [11]:
# 选择支持度的前20%分为作为阈值
min_support = df_support['support'].quantile(0.2)
min_support

0.03197359878814109

In [12]:
# 生成频繁项集
frequent_itemsets = apriori(df_te, min_support=min_support, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.114153,(三角网架)
1,0.074443,(击打手套)
2,0.047338,(垒垫)
3,0.115938,(垒球)
4,0.322441,(头盔)
5,0.115343,(帽子)
6,0.459316,(棒球手套)
7,0.172690,(棒球服)
8,0.246051,(球棒与球棒袋)
9,0.055129,(皮带)


In [21]:
# 生成关联规则
rules = (
    association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)
    .query('lift > 1')
    .sort_values(by='lift', ascending=False)
)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3,"(垒球, 头盔)",(软式棒球),0.057239,0.221218,0.036626,0.639887,2.892556,1.0,0.023964,2.162601,0.694009,0.151454,0.537594,0.402726
0,(垒球),(软式棒球),0.115938,0.221218,0.067410,0.581428,2.628299,1.0,0.041762,1.860568,0.700772,0.249900,0.462530,0.443074
2,"(垒球, 软式棒球)",(头盔),0.067410,0.322441,0.036626,0.543339,1.685079,1.0,0.014891,1.483723,0.435943,0.103691,0.326020,0.328465
4,"(软式棒球, 头盔)",(硬式棒球),0.075092,0.346083,0.039277,0.523055,1.511356,1.0,0.013289,1.371052,0.365812,0.102847,0.270633,0.318273
5,"(球棒与球棒袋, 软式棒球)",(硬式棒球),0.067410,0.346083,0.034029,0.504815,1.458654,1.0,0.010700,1.320552,0.337164,0.089678,0.242741,0.301571
1,(软式棒球),(硬式棒球),0.221218,0.346083,0.111069,0.502079,1.450746,1.0,0.034509,1.313294,0.398956,0.243448,0.238556,0.411505


In [22]:
for _,j in rules.iterrows():
    X = j['antecedents']
    Y = j['consequents']
    x = ','.join([items for items in X])
    y = ','.join([items for items in Y])
    print(f'{x} -> {y}')

垒球,头盔 -> 软式棒球
垒球 -> 软式棒球
垒球,软式棒球 -> 头盔
软式棒球,头盔 -> 硬式棒球
球棒与球棒袋,软式棒球 -> 硬式棒球
软式棒球 -> 硬式棒球


In [23]:
df_r = pd.DataFrame()
df_r['规则'] = rules.apply(lambda row: f"{', '.join(row['antecedents'])} -> {', '.join(row['consequents'])}", axis=1)
df_r['提升度'] = rules['support'].round(4)
df_r['置信度'] = rules['confidence'].round(4)
df_r['lift'] = rules['lift'].round(4)
df_r.reset_index(drop=True, inplace=True)
df_r

,规则,提升度,置信度,lift
0,"垒球, 头盔 -> 软式棒球",0.0366,0.6399,2.8926
1,垒球 -> 软式棒球,0.0674,0.5814,2.6283
2,"垒球, 软式棒球 -> 头盔",0.0366,0.5433,1.6851
3,"软式棒球, 头盔 -> 硬式棒球",0.0393,0.5231,1.5114
4,"球棒与球棒袋, 软式棒球 -> 硬式棒球",0.0340,0.5048,1.4587
5,软式棒球 -> 硬式棒球,0.1111,0.5021,1.4507


总结

1. 购买头盔和垒球的用户，购买软式棒球的概率为64%，并且关联性很强，推荐进行组合销售。
2. 购买垒球的用户，购买软式棒球的概率为58.1%，并且关联性很强，适合进行捆绑销售或优惠策略。
3. 购买垒球和软式棒球的用户，有一定概率会购买头盔，但是关联度相对较弱。
4. 购买头盔和软式棒球的用户，有一定概率会够来硬式棒球，但是关联度相对较弱。
5. 购买球棒与球棒袋和软式棒球的用户，有一定概率会购买硬式棒球，但是关联度较弱。
6. 购买软式棒球的用户，有50%的概率会买硬式棒球，但影响力较低。